In [1]:
# Kalayaporn Wangsthaporn
# Assignment 6
# Machine Learning
# Neural Network

In [59]:
import numpy as np                                       
import pandas as pd                                      
from sklearn.ensemble import RandomForestRegressor      
from sklearn.metrics import mean_squared_error as mse    
from sklearn.model_selection import train_test_split     
from sklearn.preprocessing import StandardScaler         
from keras.models import Sequential                      
from keras.layers import Dense                          

In [60]:
cdsdata = pd.read_stata("/Users/kalayaporn/Desktop/cds_spread5y_2001_2016.dta")

In [61]:
cdsdata.shape

(169043, 9)

In [62]:
cdsdata.dtypes

gvkey                    object
mdate            datetime64[ns]
redcode                  object
tier                     object
ticker                   object
shortname                object
docclause                object
spread5y                float64
impliedrating            object
dtype: object

In [63]:
cdsdata.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB


In [64]:
qmcdata = pd.read_csv("/Users/kalayaporn/Desktop/Quarterly Merged CRSP-Compustat.csv", low_memory = False)

In [65]:
qmcdata.shape

(58928, 687)

In [66]:
qmcdata.dtypes

GVKEY          int64
LINKPRIM      object
LIID          object
LINKTYPE      object
LPERMNO        int64
LPERMCO        int64
LINKDT        object
LINKENDDT     object
datadate      object
fyearq         int64
fqtr           int64
fyr            int64
indfmt        object
consol        object
popsrc        object
datafmt       object
tic           object
cusip         object
conm          object
acctchgq      object
acctstdq      object
adrrq        float64
ajexq        float64
ajpq         float64
bsprq         object
compstq       object
curcdq        object
curncdq       object
currtrq      float64
curuscnq     float64
              ...   
addzip        object
busdesc       object
city          object
conml         object
county        object
dlrsn        float64
ein           object
fax           object
fyrc           int64
ggroup         int64
gind           int64
gsector        int64
gsubind        int64
idbflag       object
incorp        object
loc           object
naics        

In [67]:
qmcdata.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyearq,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2001,2000,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
1,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,06/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
2,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,09/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
3,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,12/31/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
4,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2002,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN


In [68]:
# cdsdata
# Convert mdate to datetime format
# Seperate Month/ Year 

cdsdata['Date'] = pd.to_datetime(cdsdata['mdate'])
cdsdata['Month'] = cdsdata['Date'].dt.month
cdsdata['Year'] = cdsdata['Date'].dt.year

In [69]:
cdsdata.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating,Date,Month,Year
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB,2004-08-31,8,2004
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB,2005-08-31,8,2005
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB,2005-09-30,9,2005
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB,2005-10-31,10,2005
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB,2005-11-30,11,2005


In [70]:
#Setting the different quarters in cdsdata
cdsdata['Quarter'] = '4'

#Convert month into 'Quarter'
#For quarter 4
#Month 10 - 12
cdsdata['Quarter'][cdsdata['Month'] > 9] = '4'

#For quarter 3 
#Month 7 - 9
cdsdata['Quarter'][(cdsdata['Month'] > 6) & (cdsdata['Month'] < 9)] = '3'

#For quater 2
#Month 4 - 6
cdsdata['Quarter'][(cdsdata['Month'] > 3) & (cdsdata['Month'] < 6)] = '2'

#For quater 1
#Month 1 - 3
cdsdata['Quarter'][cdsdata['Month'] < 3] = '1'


/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/

In [71]:
cdsdata.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating,Date,Month,Year,Quarter
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB,2004-08-31,8,2004,3
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB,2005-08-31,8,2005,3
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB,2005-09-30,9,2005,4
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB,2005-10-31,10,2005,4
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB,2005-11-30,11,2005,4


In [72]:
# Transform the entity type in cdsdata to float

cdsdata['gvkey'] = cdsdata['gvkey'].astype(float)
cdsdata['Quarter'] = cdsdata['Quarter'].astype(float)
cdsdata['Year'] = cdsdata['Year'].astype(float)

In [73]:
# Change columns names in qmcdata to match colums in cdsdata

qmcdata = qmcdata.rename(columns = {'datadate':'mdate'})
qmcdata = qmcdata.rename(columns = {'GVKEY':'gvkey'})

In [74]:
qmcdata.head()

,gvkey,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,mdate,fyearq,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2001,2000,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
1,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,06/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
2,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,09/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
3,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,12/31/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
4,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2002,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN


In [75]:
# qmcdata
# Convert mdate to datetime format
# Seperate Month/ Year 

qmcdata['Date'] = pd.to_datetime(qmcdata['mdate'])
qmcdata['Month'] = qmcdata['Date'].dt.month
qmcdata['Year'] = qmcdata['Date'].dt.year

In [76]:
#Setting the different quarters in qmcdata
qmcdata['Quarter'] = '4'

#Convert month into 'Quarter'
#For quarter 4
#Month 10 - 12
qmcdata['Quarter'][qmcdata['Month'] > 9] = '4'

#For quarter 3
#Month 7 - 9
qmcdata['Quarter'][(qmcdata['Month'] > 6) & (qmcdata['Month'] < 9)] = '3'

#For quater 2
#Month 4 - 6
qmcdata['Quarter'][(qmcdata['Month'] > 3) & (qmcdata['Month'] < 6)] = '2'

#For quater 1
#Month 1 - 3 
qmcdata['Quarter'][qmcdata['Month'] < 3] = '1'

/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/kalayaporn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/

In [77]:
# Transform the columns in qmcdata to float

qmcdata['gvkey'] = qmcdata['gvkey'].astype(float)
qmcdata['Quarter'] = qmcdata['Quarter'].astype(float)
qmcdata['Year'] = qmcdata['Year'].astype(float)

In [78]:
# Merge cdsdata and qmcdata 
# Use 'gvkey' ,'Quarter' , 'Year'

mergedata = pd.merge(qmcdata, cdsdata, on=['gvkey', 'Quarter','Year'])

In [79]:
mergedata.head()

,gvkey,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,mdate_x,fyearq,...,mdate_y,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating,Date_y,Month_y
0,1045.0,P,01,LC,21020,20010,01/31/1962,01/04/2012,03/31/2002,2002,...,2002-03-31,0A18DB,SNRFOR,AMR,AMR Corp,MR,0.043750,CCC,2002-03-31,3
1,1045.0,P,01,LC,21020,20010,01/31/1962,01/04/2012,03/31/2002,2002,...,2002-06-30,0A18DB,SNRFOR,AMR,AMR Corp,MR,0.047925,CCC,2002-06-30,6
2,1045.0,P,01,LC,21020,20010,01/31/1962,01/04/2012,03/31/2002,2002,...,2002-09-30,0A18DB,SNRFOR,AMR,AMR Corp,MR,0.220847,CCC,2002-09-30,9
3,1045.0,P,01,LC,21020,20010,01/31/1962,01/04/2012,03/31/2002,2002,...,2002-10-31,0A18DB,SNRFOR,AMR,AMR Corp,MR,0.269216,CCC,2002-10-31,10
4,1045.0,P,01,LC,21020,20010,01/31/1962,01/04/2012,03/31/2002,2002,...,2002-11-30,0A18DB,SNRFOR,AMR,AMR Corp,MR,0.254322,CCC,2002-11-30,11


In [80]:
# Extract only numerical records 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [81]:
# Mergedata without non-numerical data
mergedata = mergedata.select_dtypes(include=numerics)

In [82]:
# Pre processing data
# Replace NA with median 
mergedata=mergedata.fillna(mergedata.median())

In [83]:
# Remove missing value variable
mergedata=mergedata.dropna(axis=1, how='any')

In [84]:
mergedata.head()

,gvkey,LPERMNO,LPERMCO,fyearq,fqtr,fyr,adrrq,ajexq,ajpq,currtrq,...,priusa,sic,spcindcd,spcseccd,stko,Month_x,Year,Quarter,spread5y,Month_y
0,1045.0,21020,20010,2002,1,12,1.0,1.0,1.0,1.0,...,4,4512,605.0,600.0,0.0,3,2002.0,4.0,0.043750,3
1,1045.0,21020,20010,2002,1,12,1.0,1.0,1.0,1.0,...,4,4512,605.0,600.0,0.0,3,2002.0,4.0,0.047925,6
2,1045.0,21020,20010,2002,1,12,1.0,1.0,1.0,1.0,...,4,4512,605.0,600.0,0.0,3,2002.0,4.0,0.220847,9
3,1045.0,21020,20010,2002,1,12,1.0,1.0,1.0,1.0,...,4,4512,605.0,600.0,0.0,3,2002.0,4.0,0.269216,10
4,1045.0,21020,20010,2002,1,12,1.0,1.0,1.0,1.0,...,4,4512,605.0,600.0,0.0,3,2002.0,4.0,0.254322,11


In [85]:
# Data Spliting
# assign value to X and Y
X = mergedata.drop('spread5y', axis = 1)
y = mergedata['spread5y']

In [86]:
# Split the data into 80% train and 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.head()

,gvkey,LPERMNO,LPERMCO,fyearq,fqtr,fyr,adrrq,ajexq,ajpq,currtrq,...,naics,priusa,sic,spcindcd,spcseccd,stko,Month_x,Year,Quarter,Month_y
60592,6338.0,24053,21035,2006,4,12,1.0,1.0,1.0,1.0000,...,22111,1,4931,705.0,700.0,0.0,12,2006.0,4.0,9
54845,5742.0,21792,20939,2010,1,12,1.0,1.0,1.0,1.0000,...,22111,1,4931,705.0,700.0,0.0,3,2010.0,4.0,10
51821,5543.0,65947,2105,2006,4,12,1.0,1.0,1.0,1.0000,...,531110,1,6798,850.0,800.0,0.0,12,2006.0,4.0,6
214462,241216.0,88822,41009,2004,4,12,0.2,1.0,1.0,1.0000,...,3253,90,2870,167.0,970.0,0.0,12,2004.0,4.0,12
175498,61616.0,82637,31192,2002,3,12,5.0,2.5,2.5,0.9879,...,486210,90,2911,390.0,935.0,0.0,9,2002.0,4.0,10


In [87]:
# Feature Selection
# Remove features train dataset
X_train= X_train.drop('Month_x', axis = 1)
X_train= X_train.drop('Month_y', axis = 1)
X_train= X_train.drop('Quarter', axis = 1)
X_train= X_train.drop('Year',    axis = 1)
X_train= X_train.drop('gvkey',   axis = 1)

In [88]:
# Remove features test dataset
X_test = X_test.drop('Month_x', axis = 1)
X_test = X_test.drop('Month_y', axis = 1)
X_test = X_test.drop('Quarter', axis = 1)
X_test = X_test.drop('Year',    axis = 1)
X_test = X_test.drop('gvkey',   axis = 1)

In [89]:
scaler = StandardScaler()

# Fit only to the training data
scaler.fit(X_train)
StandardScaler(copy = True, with_mean = True, with_std = True)

X_train_1 = X_train
X_test_1 = X_test

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
# Random Forest
# n_estimators = n_tree = 10
RF = RandomForestRegressor(n_estimators = 10) 
RF.fit(X_train, y_train) 

# Score the model
RF.score(X_test, y_test)
RF_pred_10 = RF.predict(X_test)

In [91]:
# Importance Feature
ImpF = RF.feature_importances_
ImpF = pd.DataFrame(RF.feature_importances_,
                    index = X_train_1.columns,
                    columns = ['importance']).sort_values('importance',ascending=False)

In [92]:
# Keep top 40 of Importance Variables
Imp_Fea_40 = ImpF.iloc[:40, :]
Imp_Fea_40 = ImpF.index.tolist()

In [93]:
# Calculate MSE and MAPE 

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#mean_squared_error(y_test,RF_pred)
mean_absolute_percentage_error(y_test, RF_pred_10)

27.542955206562052

In [94]:
# Filter top 40 features

Filter_X_train = X_train_1[Imp_Fea_40]
Filter_X_test = X_test_1[Imp_Fea_40]

In [95]:
# Filter the fitted standardized training data

scaler.fit(Filter_X_train)
StandardScaler(copy = True, with_mean = True, with_std = True)

Filter_X_train_1 = scaler.transform(Filter_X_train)
Filter_X_test_1 = scaler.transform(Filter_X_test)

In [96]:
# Neural Network
# Generate Random Seed
np.random.seed(7)


In [97]:
# Create MLP model

model_1 = Sequential()
model_1.add(Dense(32, input_dim = 50, activation = 'relu'))
model_1.add(Dense(8, activation = 'relu'))
model_1.add(Dense(1, activation = 'linear'))

In [100]:
# Compile MLP
model_1.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

In [56]:
# Fit MLP
model_1.fit(Filter_X_train_1, y_train, epochs = 100, batch_size = 10)

ValueError: Error when checking input: expected dense_1_input to have shape (50,) but got array with shape (631,)

In [56]:
# Neural Network Model 1
Neural_network_1 = model_1.predict(Filter_X_test_1)
mape_NN1 = mape(y_test, Neural_network_1)
mape_NN1

In [61]:
# Neural Network Model 2
model_2 = Sequential()
model_2.add(Dense(32, input_dim = 50, activation = 'sigmoid'))
model_2.add(Dense(8, activation = 'sigmoid'))
model_2.add(Dense(1, activation = 'relu'))
model_2.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

modelmodel_2.fit(Filter_X_train_1, y_train, epochs = 10, batch_size = 10)

Neural_network_2 = modelmodel_2.predict(Filter_X_test_1)
mape_NN2 = mape(y_test, Neural_network_2)
mape_NN2

Mean Squared Error for Gradient Boosting with 100 trees:  0.000989272853674


In [62]:
# Neural Network Model 3
model_3 = Sequential()
model_3.add(Dense(32, input_dim = 50, activation = 'sigmoid'))
model_3.add(Dense(8, activation = 'sigmoid'))
model_3.add(Dense(1, activation = 'relu'))
model_3.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

model.fit(Filter_X_train_1, y_train, epochs = 50, batch_size = 10)

Neural_network_3 = modelmodel_3.predict(Filter_X_test_1)
mape_NN3 = mape(y_test, Neural_network_3)
mape_NN3

Mean Squared Error for Gradient Boosting  with 200 trees:  0.000921578838899
